In [174]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (30,10)
import numpy as np 
import pandas as pd 
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 

In [175]:
df_train=pd.read_csv("sales_train.csv")
df_test=pd.read_csv("test.csv")
shop=pd.read_csv("shops.csv")
items=pd.read_csv("items.csv")
item_categories=pd.read_csv("item_categories.csv")

In [176]:
df_train["date"]=pd.to_datetime(df_train["date"])
df_train['day']=df_train['date'].dt.day
df_train['month']=df_train['date'].dt.month
df_train['year']=df_train['date'].dt.year
df_train=df_train.sort_values(by='date')
df_test['month']=11
df_test['year']=2015
#df_train.drop(['date'],axis=1,inplace=True)

In [177]:
merged=pd.merge(df_train,items[['item_id','item_category_id']],on='item_id')

In [178]:
merged.drop(['date','day'],axis=1,inplace=True)

In [179]:
final_df=merged.groupby(['year','month','shop_id','item_id']).agg({'item_price': 'mean', 
                                                                   'item_cnt_day': 'sum'
                                                                  }).reset_index()

In [180]:
final_df.drop(['item_price'],axis=1,inplace=True)

In [181]:
final_df=final_df[['shop_id', 'item_id', 'month', 'year','item_cnt_day']]

## Splitting Sequentially

In [182]:
X_train=final_df.iloc[:1259576,:].drop(['item_cnt_day'],axis=1)
Y_train=final_df.iloc[:1259576,:]['item_cnt_day']
X_test= final_df.iloc[1259576:,:].drop(['item_cnt_day'],axis=1)
Y_test= final_df.iloc[1259576:,:]['item_cnt_day']

## Model Training

In [183]:
train_dmatrix = xg.DMatrix(data = X_train, label = Y_train) 
test_dmatrix = xg.DMatrix(data = X_test, label = Y_test) 

In [184]:
param = {"booster":"gblinear", "objective":"reg:linear"} 
  
xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 20) 
pred = xgb_r.predict(test_dmatrix) 

[11:45:34] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [185]:
rmse = np.sqrt(MSE(Y_test, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  9.104588


## Making Submission

In [192]:
X_train=final_df.drop(['item_cnt_day'],axis=1)
Y_train=final_df['item_cnt_day']
train_dmatrix = xg.DMatrix(data = X_train, label = Y_train) 

In [187]:
submission=pd.DataFrame()
submission['ID']=df_test['ID']
submission['item_cnt_month']=predictions

In [194]:
submission.to_csv('Round 1| XGB | No Item Cat.csv',index=False)
